Import related libraries

In [ ]:
import openai
# sokoban_solver is a custom package that contains the A* agent that is able to solve Sokoban environment levels
from src.utilities import *

You must add your own OpenAI API key to the `openai-api-key.txt` file.

In [ ]:
api_key = get_file_contents("openai-api-key.txt")

Initialize the OpenAI API Client with the API key

In [ ]:
openai.api_key = api_key

Set your training mode for the fine-tuning process, all files related to this mode will have the name of the mode in their names.

In [ ]:
train_mode = "training-set-10-examples"

Upload the training file to the API for fine-tuning purposes

In [ ]:
training_data_path = f"data/training_data/{train_mode}.jsonl"

with open(training_data_path, "r") as file:
    response = openai.File.create(file=file, purpose="fine-tune")

file_id = response["id"]
write_file_contents(f"data/client_file_ids/{train_mode}.txt", file_id)

Set the model to fine-tune. gpt-3.5-turbo-0125 is the currently recommended model.

In [ ]:
model = "gpt-3.5-turbo-0125"

Create a fine-tuning job for the model with the training file

In [ ]:
training_file_id = get_file_contents(f"data/client_file_ids/{train_mode}.txt")

response = openai.FineTune.create_job(model=model, training_file_id=training_file_id, suffix=train_mode)

fine_tuning_job_id = response["id"]
write_file_contents(f"data/fine_tuning_job_ids/{train_mode}.txt", fine_tuning_job_id)